In [15]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd()
print('Current working directory: ' + cwd + '\n')
      
relativepath = os.path.join('..', 'data', 'fivethirtyeight', 'bechdel.csv')
bechdel = pd.read_csv(relativepath)

# bringing functions over from in-class exercises
def dot(vectorA, vectorB):
    ''' Calculates the dot product of two vectors; in other words, the
    sum of the pairwise products of A and B.
    '''
    assert len(vectorA) == len(vectorB)
    # If that's not true, this function should just fail.
    n = len(vectorA)
    dotproduct = 0    
    for i in range(n):
        dotproduct += vectorA[i] * vectorB[i]
        
    return dotproduct
    # this is the value the function will return

def covar(vecA, vecB):
    assert len(vecA) == len(vecB)
    n = len(vecA)
    diffA = vecA - np.mean(vecA)
    diffB = vecB - np.mean(vecB)
    cov = dot(diffA, diffB) / (n-1)
    
    return cov
        
def pearson(vec1, vec2):
    x = covar(vec1, vec2)
    stdev = x / (np.std(vec1) * np.std(vec2))
    return stdev

Current working directory: /Users/rdubnic2/Documents/lis590dsh/Code



## Remaining In-Class Exercises

### Exercise 6

Write Python code to test the significance of a correlation between two arbitrary vectors. It should return r, the correlation coefficient, as well as p, the fraction of times we get a larger correlation than r out of (say) a hundred tries with a randomly-permuted relationship between the vectors.

In [16]:
# Code for Exercise 6

# I'm not entirely sure what is being asked here, as it's hard to tell from the wording of the question,
#  especially given my near complete lack of mathematical knowledge. But this is my shot:

def randcorr(vecX, vecY):
    import random
    x = pearson(vecX, vecY)
    print('r = ', + x)
    count = 0
    for x in range(50):
        random.shuffle(vecX)
        # print(vecX)
        if pearson(vecX, vecY) > x:
            count += 1
        else:
            count += 0
        # print(count)
    for y in range(50):
        random.shuffle(vecY)
        # print(vecY)
        if pearson(vecX, vecY) > x:
            count += 1
        else:
            count += 0
        # print(count)
    print('p = ', + count)

vecC = [43,1.6,71,12.3,5.7]
vecD = [0.875,7.1,.888,9.987,11]

randcorr(vecC, vecD)

r =  -1.0857779238
p =  0


## Exercise 7

Use the function you've defined to test the significance of correlations between budget and international gross returns for our movies, and then also between budget and ROI.

You may need to remove rows in the dataframe where gross returns are missing. The ```isnull()``` function will help.

In [17]:
# Code for Exercise 7

# print(bechdel['intgross_2013'].isnull())
adjbech = bechdel.fillna(0)
adjbech.head()

# randcorr(adjbech['budget_2013'], adjbech['intgross_2013'])
# randcorr(adjbech['budget_2013'], adjbech['roi'])

,year,imdb,title,test,clean_test,binary,budget,domgross,intgross,code,budget_2013,domgross_2013,intgross_2013,period_code,decade_code
0,2013,tt1711425,21 & Over,notalk,notalk,FAIL,13000000,25682380.0,42195766.0,2013FAIL,13000000,25682380.0,42195766.0,1.0,1.0
1,2012,tt1343727,Dredd 3D,ok-disagree,ok,PASS,45000000,13414714.0,40868994.0,2012PASS,45658735,13611086.0,41467257.0,1.0,1.0
2,2013,tt2024544,12 Years a Slave,notalk-disagree,notalk,FAIL,20000000,53107035.0,158607035.0,2013FAIL,20000000,53107035.0,158607035.0,1.0,1.0
3,2013,tt1272878,2 Guns,notalk,notalk,FAIL,61000000,75612460.0,132493015.0,2013FAIL,61000000,75612460.0,132493015.0,1.0,1.0
4,2013,tt0453562,42,men,men,FAIL,40000000,95020213.0,95020213.0,2013FAIL,40000000,95020213.0,95020213.0,1.0,1.0


## Homework 3: Functions and Hypothesis-Testing

This homework begins by reviewing some things we accomplished in class, and defining some functions. Then it asks you to apply those functions to the Bechdel dataset. We won't fully replicate the data analysis performed in [the Bechdel article,](https://fivethirtyeight.com/features/the-dollar-and-cents-case-against-hollywoods-exclusion-of-women/) and spelled out (using R) in [this online analysis.](https://mran.microsoft.com/web/packages/fivethirtyeight/vignettes/bechdel.html) But we're going to build some elements of the process from the ground up, so that we genuinely understand what's going on — especially where hypothesis testing is involved.

I should say up front that the approach we're taking to hypothesis-testing here is only one of several possible approaches; it's based on what statisticians call a *frequentist* theory of statistics. *Bayesian* statistics provide an alternative way of reasoning about these questions; we'll glance at that in a couple of weeks.

To start with, let's test the significance of a correlation. We saw in class that movies that fail to pass the Bechdel test have lower median return-on-investment than those that do pass. This hints that movie studios may be controlled by a kind of sexism that is not only unjust, and not only not profitable, but actually self-destructive and money-*losing.* 

On the other hand, I mentioned in class that there's a potential *confounding variable* in this argument. Movies with lower budgets generally have higher ROI. So perhaps movies that pass the Bechdel test have higher ROI simply *because* they have lower budgets? We won't completely resolve that question below, but we'll build up to it. To begin with, is the relationship between budget and return-on-investment actually meaningful, or would we be likely to see relations of that kind emerge by chance?

To test that, we'll start by reconstructing the functions we need for a correlation test.

In [51]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# We start (above) by importing the usual suspects.

# Read the functions below to get a sense of how you can build up a complex
# function by breaking it into elements.

def dot(vectorA, vectorB):
    ''' Calculates the dot product of two vectors; in other words, the
    sum of the pairwise products of A and B.
    '''
    assert len(vectorA) == len(vectorB)
    dotproduct = np.sum(vectorA * vectorB)
    # that's the simpler version of a dot product, using
    # numpy's automatic elementwise multiplication of vectors
    
    return dotproduct

def covariance(vecA, vecB):
    ''' Calculates the covariance of two vectors; in other words, the
    tendency for A to be above its average when B is also above average,
    and below when B is also below.
    '''
    
    n = len(vecA)
    if n > 1:
        diffA = vecA - np.mean(vecA)
        diffB = vecB - np.mean(vecB)
        cov = dot(diffA, diffB) / n
        
        # In class we were using n-1 here, which gives sample covariance.
        # The official function uses n (population covariance), so that's
        # what I've substituted. It won't make much difference as we scale up
        # to large n.
        
        return cov
    else:
        print('Error! covariance has no meaning if vectors have less than two elements.')
        return float('NaN')

def pearson_correlation(vecA, vecB):
    ''' Calculates the Pearson correlation coefficient, r, for two
    vectors.
    '''
    
    cov = covariance(vecA, vecB)
    
    if np.std(vecA) == 0 or np.std(vecB) == 0:
        print('Error! statistic undefined if standard deviations are zero.')
        return float('NaN')
    else:
        pearson = cov / (np.std(vecA) * np.std(vecB))
        return pearson

    
vectorA = np.array([1, 1, 3, 4, 5, 2, 7, 8, 7])
vectorB = np.array([0, 2, 4, 5, 2, 6, 8, 9, 6])

print('Our version: ', pearson_correlation(vectorA, vectorB))

# Let's test that we're getting this roughly right

from scipy.stats import pearsonr

print('Official version: ', pearsonr(vectorA, vectorB))

    

Our version:  0.766723820184
Official version:  (0.76672382018354224, 0.015931317092922143)


## An example of what it means to calculate a p-value

It appears that we're calculating Pearson's r the same way the scipy module does it. But they provide an additional piece of information: the p-value, or the number of times you would get an equally large (or small!) r value if the data was random, with no relation between the vectors. Let's test that in an intuitive way.

We'll randomize the data and see how often we do get an equally large (or small) r value. Here's how you can randomize a list (or a numpy vector / pandas Series):

In [19]:
import random
vectorC = [1, 2, 3, 4, 5]
random.shuffle(vectorC)
print(vectorC)

[3, 2, 1, 4, 5]


Now we need to repeatedly test correlation between our two vectors while randomizing one of them. The number of random correlations that are equal to or greater than our r value will tell us how often this could happen by chance.

A Pearson's correlation coefficient can range from -1 (perfect inverse correlation) to 1 (perfect positive correlation). We'll use the *absolute value* of all the r values (i.e, forcing them all to be positive), because we didn't actually start with a hypothesis about whether this correlation was negative or positive. So *any* strong correlation (say an inverse correlation of -0.77) should count as evidence that "a statistic this extreme could have occurred by chance." To test absolute magnitude rather than sign, we'll transform everything using the ```abs()``` function.


In [52]:
def permutation_test_of_pearson_2tailed(vectorA, vectorB):
    r = pearson_correlation(vectorA, vectorB)
    absolute_r = abs(r)
    
    copiedA = np.array(vectorA)
    copiedB = np.array(vectorB)
    # we do that to avoid permanently shuffling vectorA
    
    random_r_values = list()
    
    number_of_tests = 1000
    
    for i in range(number_of_tests):
        random.shuffle(copiedA)
        random.shuffle(copiedB)
        random_r = pearson_correlation(copiedA, copiedB)
        random_r_values.append(abs(random_r))
 
    # how many random r values are greater than or equal to the
    # one we found in the actual vectors?
    # to find out, we'll use the enumerate function,
    # which returns elements of a list paired with
    # their numeric index in the list
    
    random_r_values.sort(reverse = True)
    for index, random_val in enumerate(random_r_values):
        if random_val < absolute_r:
            break
    p = index / number_of_tests
    
    return r, p
              
permutation_test_of_pearson_2tailed(vectorA, vectorB)

(0.76672382018354246, 0.015)

The p value calulated by our function won't be precisely equal to the value calculated by pearsonr, because there's an element of randomness here. But they will usually be in the same ballpark

## Problem 1: Editing the permutation test.

Suppose we had started by predicting that the correlation between these two vectors would be *positive.* Then, in principle, we would only want to know how often a raw correlation coefficient greater than the observed value could occur by chance. Random negative correlations would not do anything to weaken our confidence.

Copy and paste the code of the 2-tailed permutation test, and then edit it to make it 1-tailed. In other words, now you only want to ask how often the random Pearson's correlation is actually *larger* than the observed value, not how often it's *more extreme* than the observed value.

If this is opaque, you might want to read [this account of the difference between one- and two-tailed tests.](https://en.m.wikipedia.org/wiki/One-_and_two-tailed_tests)

The edit required is actually very simple; I just want to give you an occasion to read through the function and understand how it works.

In [53]:
# Code for problem 1.

def permutation_test_of_pearson_1tailed(vectorA, vectorB):
    r = pearson_correlation(vectorA, vectorB)
    # removed use of abs() to force r positive
    copiedA = np.array(vectorA)
    copiedB = np.array(vectorB)
    # we do that to avoid permanently shuffling vectorA
    
    random_r_values = list()
    
    number_of_tests = 1000
    
    for i in range(number_of_tests):
        random.shuffle(copiedA)
        random.shuffle(copiedB)
        random_r = pearson_correlation(copiedA, copiedB)
        random_r_values.append(random_r) # removed abs() to force appended r values to be positive
 
    # how many random r values are greater than or equal to the
    # one we found in the actual vectors?
    # to find out, we'll use the enumerate function,
    # which returns elements of a list paired with
    # their numeric index in the list
    
    random_r_values.sort(reverse = True)
    for index, random_val in enumerate(random_r_values):
        if random_val < r: # replaced call to abs(r) with just r, to reflect potential negatives
            break
    p = index / number_of_tests
    
    return r, p
              
permutation_test_of_pearson_1tailed(vectorA, vectorB)

(0.76672382018354246, 0.012)

## Problem 2: Significance of correlation between budget and ROI

Read in the Bechdel dataset (using the code below). Note that this code filters the dataset in several ways: for instance, it drops all rows where NaNs occur. I've done this for you using the .dropna() method. In this case, we've only dropped rows where NaNs occur in columns we're going to use (that's the significance of the "subset" argument).

I also drop all films before 1990, in order to replicate the analysis at https://mran.microsoft.com/web/packages/fivethirtyeight/vignettes/bechdel.html.

In [23]:
cwd = os.getcwd()
print('Current working directory: ' + cwd + '\n')
      
relativepath = os.path.join('..', 'data', 'fivethirtyeight', 'bechdel.csv')
bechdel = pd.read_csv(relativepath)
print("Original shape: ", bechdel.shape)
bechdel = bechdel.dropna(subset = ['domgross_2013', 'budget_2013', 'intgross_2013'])
print("After dropping rows with NaN: ", bechdel.shape)
bechdel = bechdel[bechdel['year'] > 1989]
print("After dropping rows before 1990: ", bechdel.shape)

Current working directory: /Users/rdubnic2/Documents/lis590dsh/Code

Original shape:  (1794, 15)
After dropping rows with NaN:  (1776, 15)
After dropping rows before 1990:  (1600, 15)


Now let's figure out whether it's really meaningfully true that films with bigger budgets have lower return-on-investment. 

To figure this out, you'll need to start by creating a return-on-investment column. For our purposes let's focus on *domestic* ROI, which we'll calculate as the domestic gross receipts (inflation-corrected to 2013 dollars) divided by the film's budget (also inflation-corrected to 2013 dollars).

Let's also adopt the conventional definition of statistical significance as p < 0.05, or roughly "if the null hypothesis were true, a test statistic equally extreme could have occurred by chance more than 5% of the time."

Given that definition, find out whether domestic return on investment has a significant (positive or negative) correlation with budget. Use 2013 dollars in both cases. In principle, you could test significance of the correlation using *either* the official ```pearsonr``` function or our ```permutation_test_of_pearson_2tailed``` function. But use both, to see how well they agree.

In [68]:
# Code for problem 2

bechdel['domroi_2013'] = (bechdel['domgross_2013'] / bechdel['budget_2013'])
bechdel.head()

print('Our version: ', permutation_test_of_pearson_2tailed(bechdel['domroi_2013'], bechdel['budget_2013']))

print('Official version: ', pearsonr(bechdel['domroi_2013'], bechdel['budget_2013']))

Our version:  (-0.11954316394556917, 0.0)
Official version:  (-0.1195431639455691, 1.6251759110499532e-06)


There does appear to be a correlation between domestic ROI and budget size for films post-1990. It appears to be a slight negative correlation, giving credence to the idea that films with larger budgets have a slight tendency to have smaller ROIs.

## Problem 3: Difference between medians

Let's also test the significance of the difference we observed between median ROI for films that do or don't pass the Bechdel test. 

First, calculate the median domestic return on investment for films that do, or don't, pass the Bechdel test (using the ```binary``` column to divide them).

Then import the function ```median_test``` from ```scipy.stats``` (the same place we got pearsonr), and use it to assess the significance of the difference between medians. 

For a [full explanation of that function,](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.median_test.html) see the scipy documentation. You'll need to read this in order to figure out, for instance, how many arguments to send to the function.

Basically, this function is doing something analogous to our permutation-test for significance of correlations. It's pooling all the data (both vectors of ROI figures, for films that pass the Bechdel test and films that fail) and calculating a "grand median" for the whole group. Then it asks how often we would randomly draw two groups of films from that master set that are as differently-distributed relative to the "grand median" as the two groups we actually passed in as arguments.

It may be a useful reminder that when you index a DataFrame with single label like ```bechdel['budget_2013']```, you're selecting a column. But when you index using a list or vector of Boolean true/false values ```bechdel[bechdel['binary'] == 'PASS']```, you're selecting rows where that condition is true.


In [92]:
# Code for problem 3

grouped_bechdel = bechdel.groupby('binary')
bmean = grouped_bechdel.aggregate(np.mean)['domroi_2013']
bmedian = grouped_bechdel.aggregate(np.median)['domroi_2013']
bcount = grouped_bechdel.count()['domroi_2013']

from scipy.stats import median_test
bechpass = bechdel[bechdel['binary'] == 'PASS']
bechfail = bechdel[bechdel['binary'] == 'FAIL']

median_test(bechpass['domroi_2013'], bechfail['domroi_2013'])


(9.3483067028439351,
 0.0022319158626105144,
 1.2869639280222738,
 array([[403, 397],
        [341, 459]]))

## Problem 4: Discussion and conclusions.

We haven't yet answered the question we started out with. To work it out completely in this assignment would require a few tricks I haven't yet taught you using Pandas, but let's leap forward a bit by relying on [the published analysis of this data I mentioned before.](https://mran.microsoft.com/web/packages/fivethirtyeight/vignettes/bechdel.html)

This will be a little tricky to read, because they use R rather than Python/Pandas. But the principles are the same, and the significance of p-values, for instance, should now be legible to you, even if the format is a little different.

How much can we infer from this evidence about sexism in the movie industry?
Do you notice any important data analysis tricks used in the published analysis that we haven't covered yet?

```Write your conclusions in this Markdown cell. A paragraph of 5-8 sentences should be fine.```

## _Conclusions_

I'm not entirely sure we can infer a huge amount about sexism in the movie industry from this analysis, to be honest, but there are some very general findings that are concluded. It looks like movies with larger budgets do indeed tend to have a smaller ROI, and films that pass the Bechdel test do tend to have smller budgets. This could possibly mean either that studios/financiers are less likely to finance films that pass the Bechdel test, that films that pass the test are less likely to opt for studio financing, or even that they tend to be in genres that are less budget-intensive (i.e. drama, comedy, romance, etc.). Because of this, it's still unclear whether films with larger budgets ensured they passed the test would see larger ROIs. However, we are seeing that this effect only applies to U.S. and Canada ROIs, with international ROI evening out over the long-term.

All-in-all, it's likely fair to say that films that pass the Bechdel test do not have the level of funding available to them as those that do not (for whatever reason), and still tend to do very well in ROI. Hollywood is clearly _very_ sexist, in my opinion, but I'm not sure a lot of specifics can be pulled from this beyond these two conclusions.

The most interesting trick in the linked analysis is breaking out a return on investment per dollar spent figure, which really clarifies this analysis, for me. Rather than dealing with means and mediand directly, then comparing where the films fall, it's a much more elegant and simple comparison to show one numerical figure to illustrate all of the arguments. Using their plot style/mechanism is also helpful for identifying the distribution of the data.